In [142]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import Plotting Libararies
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Model Evaluation Libraries
from sklearn.metrics import classification_report, confusion_matrix

In [143]:
#expanding the dispay of text sms column
pd.set_option('display.max_colwidth', -1)

dataset = pd.read_csv("../data/high_low.csv")
dataset.head()

number  result  \
0  news_3   1        
1  news_7   1        
2  news_10  1        
3  news_14  1        
4  news_16  1        

                                                                                                                    Title  \
0  Deadline to make trains accessible for disabled ripped up in Boris Johnson's first 'broken promise' since election win   
1  Are Democrats Willing to Pay More in Taxes for Medicare for All?                                                         
2  Ghosts of Iraq hang over Trump’s chest-thumping on Iran                                                                  
3  Brexit: May loses grip on deal after fresh Commons humiliation                                                           
4  Housing Works Is Cast in Unusual Role: Corporate Overlord                                                                

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [144]:
dataset.columns

Index(['number', 'result', 'Title', 'body_x', 'date', 'result.1', 'title',
       'body_y', 'author', 'url', 'Noimages', 'Publication', 'URL', 'Message',
       'Total Interactions', 'Likes', 'Comments', 'Shares', 'Love', 'Wow',
       'Haha', 'Sad', 'Angry', 'Care',
       'Overperforming Score (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )',
       'tokenized', 'num_wds', 'difficulty', 'uniq_wds', 'lemmatized',
       'char_count', 'avg_word', 'perc_uniq', 'avg_sent', 'stopwords',
       'mistakes', 'numbers', 'first_person', 'rare', 'mtld', 'msttr', 'hdd',
       'yule', 'caps_body', 'anger', 'positivity', 'joy', 'disgust',
       'surprise', 'trust', 'anticipation', 'sadness', 'negative', 'fear',
       'density', 'sad', 'happy', 'inspired', 'dont_care', 'annoyed', 'amused',
       'afraid', 'polarity', 'negativity_vader', 'neutrality_vader',
       'positivity_vader', 'subjectivity', 'pos', 'names', 'celebrities',
       'No Celebs', 

In [145]:
data = dataset[['tokenized', 'result']]

In [146]:
data = data.dropna()

In [147]:
# Combining all the above stundents 
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os
from bs4 import BeautifulSoup

import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(dataset['tokenized'].values):
    sentance = re.sub(r"http\S+", "", str(sentance))
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = re.sub(r'\d', '', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords and not e.isnumeric())
    
    # Filter sentance to only include elements that are either byte or Unicode strings
    filtered_sentance = []
    for x in sentance:
        if isinstance(x, (bytes, str)):
            filtered_sentance.append(x)
    sentance = ''.join(filtered_sentance)
    
    preprocessed_reviews.append(sentance.strip())


100%|██████████| 1062/1062 [00:01<00:00, 640.32it/s]


In [148]:
data = data.reset_index(drop=True)

In [149]:
# create dataset
X = data['tokenized']
Y = data['result']
print(X.shape)
print(Y.shape)



(1060,)
(1060,)


# Feature Extration Using Bag of Words Model

In [150]:
#BoW
vectorizer = CountVectorizer() #in scikit-learn
X_bow = vectorizer.fit(preprocessed_reviews)
print("some feature names ", vectorizer.get_feature_names()[:10])
print('='*50)

final_counts = vectorizer.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])


some feature names  ['aa', 'aaaand', 'aap', 'aaps', 'aaron', 'ababa', 'aback', 'abady', 'abandon', 'abandoned']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (1062, 33031)
the number of unique words  33031


In [151]:
X_filtered = [x for x in X if isinstance(x, (bytes, str))]
vectorizer = CountVectorizer(stop_words='english')
X_bow = vectorizer.fit_transform(X_filtered)

In [152]:
analyze = vectorizer.build_analyzer()


In [153]:
vectorizer.get_feature_names()


['0028',
 '01',
 '02',
 '02c',
 '03',
 '04',
 '044',
 '0619',
 '08',
 '0800',
 '0925hrs',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000page',
 '1001',
 '100anight',
 '100ayear',
 '100ft',
 '100percent',
 '100th',
 '100year',
 '101',
 '1010wins',
 '101311',
 '1014',
 '10160',
 '1018',
 '101st',
 '102',
 '102000',
 '1025f',
 '103',
 '1030',
 '10300',
 '1030pm',
 '104',
 '1040',
 '10400',
 '1040pm',
 '105',
 '10500',
 '1050am',
 '1051am',
 '105am',
 '105pm',
 '106',
 '106000',
 '107',
 '108',
 '108000',
 '1085',
 '108m',
 '109',
 '1091',
 '10a',
 '10am',
 '10an',
 '10andahalf',
 '10bn',
 '10course',
 '10day',
 '10f',
 '10foot',
 '10hour',
 '10kg',
 '10m',
 '10minute',
 '10monthold',
 '10p',
 '10pm',
 '10s',
 '10th',
 '10week',
 '10year',
 '10yearold',
 '10yearolds',
 '11',
 '110',
 '1100',
 '11000',
 '110000',
 '110k',
 '110lbs',
 '111',
 '1116am',
 '112th',
 '112yearold',
 '113',
 '1130',
 '1130am',
 '1130pm',
 '1132pm',
 '113th',
 '114',
 '114000',
 '1145',
 '1145pm',
 '1146am',
 

In [154]:
X_bow.toarray()
X_bow.shape

(1060, 34477)

# Feature Extraction Using TfIDF

In [155]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(smooth_idf=False)

In [156]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vec.fit(preprocessed_reviews)
print("some sample features(unique words in the corpus)",tf_idf_vec.get_feature_names()[0:10])
print('='*50)

tf_idf_data = tf_idf_vec.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(tf_idf_data))
print("the shape of out text TFIDF vectorizer ",tf_idf_data.get_shape())
print("the number of unique words including both unigrams and bigrams ", tf_idf_data.get_shape()[1])

some sample features(unique words in the corpus) ['abandon', 'abandoned', 'ability', 'able', 'able get', 'abortion', 'abroad', 'absence', 'absolute', 'absolutely']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (1062, 5939)
the number of unique words including both unigrams and bigrams  5939


In [157]:
#create dataframe
tf_idf_dataframe=pd.DataFrame(tf_idf_data.toarray(),columns = tf_idf_vec.get_feature_names())
print(tf_idf_dataframe)
print("\n")

      abandon  abandoned   ability      able  able get  abortion  abroad  \
0     0.0      0.0        0.000000  0.000000  0.0       0.0       0.0      
1     0.0      0.0        0.023169  0.000000  0.0       0.0       0.0      
2     0.0      0.0        0.000000  0.018176  0.0       0.0       0.0      
3     0.0      0.0        0.021331  0.000000  0.0       0.0       0.0      
4     0.0      0.0        0.000000  0.016574  0.0       0.0       0.0      
...   ...      ...             ...       ...  ...       ...       ...      
1057  0.0      0.0        0.000000  0.000000  0.0       0.0       0.0      
1058  0.0      0.0        0.053985  0.000000  0.0       0.0       0.0      
1059  0.0      0.0        0.000000  0.027846  0.0       0.0       0.0      
1060  0.0      0.0        0.000000  0.000000  0.0       0.0       0.0      
1061  0.0      0.0        0.000000  0.000000  0.0       0.0       0.0      

      absence  absolute  absolutely  ...  youre going  youre not  youth  \
0     0.0   

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer

#without smooth IDF
print("Without Smoothing:")
#define tf-idf

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1), stop_words='english') # to use only  bigrams ngram_range=(2,2)

X_filtered = [x for x in X if isinstance(x, (bytes, str))]
#transform
tf_idf_data = tf_idf_vec.fit_transform(X_filtered)
 
#create dataframe
tf_idf_dataframe=pd.DataFrame(tf_idf_data.toarray(),columns = tf_idf_vec.get_feature_names())
print(tf_idf_dataframe)
print("\n")
 
#with smooth

tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english')
tf_idf_data_smooth = tf_idf_vec_smooth.fit_transform(X)
print("With Smoothing:")
tf_idf_dataframe_smooth = pd.DataFrame(tf_idf_data_smooth.toarray(),columns=tf_idf_vec_smooth.get_feature_names())
print(tf_idf_dataframe_smooth)

Without Smoothing:
      0028   01   02  02c   03   04  044  0619   08  0800  ...  zullo  zumba  \
0     0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
1     0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
2     0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
3     0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
4     0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
...   ...   ...  ...  ...  ...  ...  ...  ...   ...  ...   ...  ...    ...     
1055  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
1056  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
1057  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
1058  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     
1059  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0   ...  0.0    0.0     

      zurich  zuñiga

# Feature Extraction Using Bigrams and Trigrams

In [159]:

# matrix of ngrams

X_filtered = [x for x in X if isinstance(x, (bytes, str))]
c_vec = CountVectorizer(stop_words='english', ngram_range=(2,2))

ngrams = c_vec.fit_transform(X_filtered)
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
X_2gram  = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'unigram/bigram'})

X_2gram

,frequency,unigram/bigram
0,356,united states
1,344,white house
2,303,new york
3,248,hong kong
4,228,mr trump
...,...,...
316637,1,02c higher
316638,1,02 percent
316639,1,01 percent
316640,1,01 away


In [160]:
c_vec = CountVectorizer(stop_words='english', ngram_range=(3,3))
# matrix of ngrams

X_filtered = [x for x in X if isinstance(x, (bytes, str))]
ngrams = c_vec.fit_transform(X_filtered)
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
X_3gram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

X_3gram 

,frequency,bigram/trigram
0,57,president donald trump
1,49,new york city
2,35,quid pro quo
3,34,green new deal
4,31,spoke condition anonymity
...,...,...
377225,1,02c higher record
377226,1,02 percent americans
377227,1,01 percent unacceptable
377228,1,01 away leicester


# Binary Classification using TFIDF vectors

In [161]:
#Naïve Bayes, Logistic Regression, Random Forest, SVM, Perceptron

In [162]:
# Machine Learning Models
from sklearn import svm  
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer

In [163]:
# split into train test sets



X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape)
print('Shape of Y_train', y_train.shape)
print('Shape of Y_test', y_test.shape)

Shape of X_train (710,)
Shape of X_test (350,)
Shape of Y_train (710,)
Shape of Y_test (350,)


In [164]:
max_feature_num = 3468

vectorizer = TfidfVectorizer(max_features=max_feature_num)

train_vecs = vectorizer.fit_transform(X_train)
test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(X_test)

In [165]:
def NB_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    GB = GaussianNB()
    GB.fit(train_vecs, y_train)

    # testing
    test_predictionGB = GB.predict(test_vecs)
    return classification_report(test_predictionGB, y_test) , confusion_matrix(test_predictionGB, y_test)


In [166]:
def LR_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    LR = LogisticRegression()
    LR.fit(train_vecs, y_train)

    # testing
    test_predictionLR = LR.predict(test_vecs)
    return classification_report(test_predictionLR, y_test) , confusion_matrix(test_predictionLR, y_test)



In [167]:
def RF_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    RF = RandomForestClassifier(n_estimators = 450, max_depth=9, random_state=43)
    RF.fit(train_vecs, y_train)

    # Testing
    test_predictionRF = RF.predict( test_vecs )
    return classification_report(test_predictionRF, y_test), confusion_matrix(test_predictionRF, y_test)

In [168]:
def SVM_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    SVM = svm.LinearSVC(max_iter=100)
    SVM.fit(train_vecs, y_train)

    # Testing
    test_predictionSVM = SVM.predict(test_vecs)
    return classification_report(test_predictionSVM, y_test), confusion_matrix(test_predictionSVM, y_test)

In [169]:
def Perceptron_classifier(train_vecs, y_train, test_vecs, y_test):

    # Training
    Per = Perceptron(tol=1e-3, random_state=0)
    Per.fit(train_vecs, y_train)

    # Testing
    test_predictionPer = Per.predict(test_vecs)
    return classification_report(test_predictionPer, y_test), confusion_matrix(test_predictionPer, y_test)


In [170]:
def KNN_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    KNN = KNeighborsClassifier()
    KNN.fit(train_vecs, y_train)

    # testing
    test_predictionKNN = KNN.predict(test_vecs)
    return classification_report(test_predictionKNN, y_test), confusion_matrix(test_predictionKNN, y_test)


In [171]:
def DT_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training
    DT = DecisionTreeClassifier()
    DT.fit(train_vecs, y_train)

    # testing
    test_predictionDT = DT.predict(test_vecs)
    return classification_report(test_predictionDT, y_test), confusion_matrix(test_predictionDT, y_test)


In [172]:
def XGB_classifier(train_vecs, y_train, test_vecs, y_test):
    # Training

    XGB = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="auc")
    XGB.fit(train_vecs, y_train)

    # testing
    test_predictionXGB = XGB.predict(test_vecs)
    return classification_report(test_predictionXGB, y_test), confusion_matrix(test_predictionXGB, y_test)


# Results

In [173]:
class_report , conf_matrix = SVM_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of SVM CLASSIFIER on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)


Results of SVM CLASSIFIER on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.87      0.89      0.88       184
           1       0.88      0.86      0.87       166

    accuracy                           0.87       350
   macro avg       0.87      0.87      0.87       350
weighted avg       0.87      0.87      0.87       350

[[164  20]
 [ 24 142]]


In [178]:
# Create an SVM classifier with a linear kernel
classifier = LinearSVC()

# Fit the classifier on the training data
classifier.fit(train_vecs, y_train)

# Get the coefficients of the features (words) in the decision function
coefficients = classifier.coef_

# Sort the coefficients in descending order
sorted_coefficients = sorted(coefficients, reverse=True)

# Get the top 10 most important words
top_words = vectorizer.inverse_transform(sorted_coefficients[:10])

# Print the top 10 most important words
print(top_words)

[array(['10', '100', '1000', ..., 'zealand', 'zelensky', 'zero'],
      dtype='<U15')]


In [181]:
# Get the coefficients of the features (words) in the decision function
coefficients = classifier.coef_

# Sort the coefficients in descending order
sorted_coefficients = sorted(coefficients, reverse=True)

# Get the top 10 most important words
top_words = vectorizer.inverse_transform(sorted_coefficients[:20])

# Print the top 10 most important words
print(top_words)

[array(['10', '100', '1000', ..., 'zealand', 'zelensky', 'zero'],
      dtype='<U15')]


In [179]:
class_report , conf_matrix = LR_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Logistic Regression Classifier on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of Logistic Regression Classifier on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       188
           1       0.86      0.86      0.86       162

    accuracy                           0.87       350
   macro avg       0.87      0.87      0.87       350
weighted avg       0.87      0.87      0.87       350

[[166  22]
 [ 22 140]]


In [184]:
# Create a logistic regression classifier
classifier = LogisticRegression()

# Fit the classifier on the training data
classifier.fit(train_vecs, y_train)

# Get the coefficients of the features (words) in the decision function
coefficients = classifier.coef_


# Sort the coefficients in descending order
sorted_coefficients = sorted(coefficients, reverse=True)

# Get the top most important words
top_words = vectorizer.inverse_transform(sorted_coefficients)

# Print the top most important words
print(top_words)


[array(['10', '100', '1000', ..., 'zealand', 'zelensky', 'zero'],
      dtype='<U15')]


In [186]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
top_words

[array(['10', '100', '1000', '100000', '11', '12', '13', '14', '15', '16',
        '17', '18', '19', '1984', '1997', '20', '200', '2000', '2001',
        '2002', '2004', '2007', '2008', '2009', '2010', '2011', '2012',
        '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
        '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '300',
        '31', '32', '35', '36', '38', '39', '40', '44', '45', '50', '500',
        '50000', '57', '60', '70', '80', '90', 'ability', 'able',
        'abortion', 'about', 'above', 'absolute', 'absolutely', 'abuse',
        'abused', 'academic', 'accept', 'accepted', 'access', 'accident',
        'according', 'account', 'accountability', 'accountable',
        'accounts', 'accusations', 'accused', 'accuser', 'accusing',
        'achieve', 'acknowledged', 'across', 'act', 'acting', 'action',
        'actions', 'active', 'activist', 'activists', 'activities',
        'activity', 'actress', 'acts', 'actual', 'actually', 'adam', 'add

In [94]:
class_report , conf_matrix = RF_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of  Random Forest on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of  Random Forest on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.97      0.86      0.91       228
           1       0.78      0.95      0.86       122

    accuracy                           0.89       350
   macro avg       0.88      0.91      0.89       350
weighted avg       0.91      0.89      0.89       350

[[196  32]
 [  6 116]]


In [95]:
class_report , conf_matrix = NB_classifier(train_vecs.todense(), y_train, test_vecs.todense(), y_test)
print('Results of Naive Bayes Classifier on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of Naive Bayes Classifier on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.72      0.88      0.79       165
           1       0.86      0.69      0.77       185

    accuracy                           0.78       350
   macro avg       0.79      0.79      0.78       350
weighted avg       0.80      0.78      0.78       350

[[145  20]
 [ 57 128]]


In [96]:
#class_report , conf_matrix = Perceptron_classifier(train_vecs, y_train, test_vecs, y_test)
#print('Results of Perceptron Classifier on TF-IDF Vectorizer\n')
#print(class_report)
#print(conf_matrix)

In [97]:
class_report , conf_matrix = KNN_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of KNN_classifier on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of KNN_classifier on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.58      0.92      0.72       128
           1       0.93      0.62      0.75       222

    accuracy                           0.73       350
   macro avg       0.76      0.77      0.73       350
weighted avg       0.81      0.73      0.73       350

[[118  10]
 [ 84 138]]


In [98]:
class_report , conf_matrix = DT_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Decision tree classifier on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of Decision tree classifier on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.87      0.83      0.85       210
           1       0.76      0.81      0.78       140

    accuracy                           0.82       350
   macro avg       0.81      0.82      0.82       350
weighted avg       0.83      0.82      0.82       350

[[175  35]
 [ 27 113]]


In [99]:
class_report , conf_matrix = XGB_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of XGB classifier on TF-IDF Vectorizer\n')
print(class_report)
print(conf_matrix)

Results of XGB classifier on TF-IDF Vectorizer

              precision    recall  f1-score   support

           0       0.95      0.86      0.90       223
           1       0.79      0.92      0.85       127

    accuracy                           0.88       350
   macro avg       0.87      0.89      0.88       350
weighted avg       0.89      0.88      0.88       350

[[192  31]
 [ 10 117]]


# Binary Classification using BOW

In [100]:
vectorizer = CountVectorizer(binary=False,ngram_range=(1,1))
train_vecs = vectorizer.fit_transform(X_train)
test_vecs = vectorizer.transform(X_test)
print (train_vecs.shape, test_vecs.shape)

(710, 29015) (350, 29015)


Note: We will only consider training dataset to define the vocabulary and use the same vocabulary to 
represent the test dataset (as test data is supposed to be hidden).

Thus we will fit our vectorizer on the training data and use it to transform the test data,

There are 8016 unique words in vocabulary.

For each review in our dataset, the Frequency of words(term-frequency) is represented through a vocabulary vector of size 8016. That’s why we have 2323 such vectors in our training-set and similarly 1145 vectors of the similar shape in our test dataset.

Note: binary=False argument means that we fill the vocabulary vector with term-frequency. If binary=True, the vocabulary vector is filled by the presence of words (1 if the word is present and 0 otherwise).



In [101]:
class_report , conf_matrix = SVM_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of SVM CLASSIFIER on BOW\n')
print(class_report)
print(conf_matrix)


Results of SVM CLASSIFIER on BOW

              precision    recall  f1-score   support

           0       0.94      0.83      0.88       230
           1       0.73      0.90      0.81       120

    accuracy                           0.85       350
   macro avg       0.84      0.86      0.84       350
weighted avg       0.87      0.85      0.85       350

[[190  40]
 [ 12 108]]


In [102]:
class_report , conf_matrix = LR_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Logistic Regression Classifier on BOW \n')
print(class_report)
print(conf_matrix)

Results of Logistic Regression Classifier on BOW 

              precision    recall  f1-score   support

           0       0.95      0.86      0.90       223
           1       0.78      0.91      0.84       127

    accuracy                           0.88       350
   macro avg       0.86      0.88      0.87       350
weighted avg       0.89      0.88      0.88       350

[[191  32]
 [ 11 116]]


In [103]:
class_report , conf_matrix = RF_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of  Random Forest on BOW \n')
print(class_report)
print(conf_matrix)

Results of  Random Forest on BOW 

              precision    recall  f1-score   support

           0       0.94      0.83      0.88       227
           1       0.74      0.89      0.81       123

    accuracy                           0.85       350
   macro avg       0.84      0.86      0.85       350
weighted avg       0.87      0.85      0.86       350

[[189  38]
 [ 13 110]]


In [104]:
class_report , conf_matrix = NB_classifier(train_vecs.todense(), y_train, test_vecs.todense(), y_test)
print('Results of Naive Bayes Classifier on BOW \n')
print(class_report)
print(conf_matrix)

Results of Naive Bayes Classifier on BOW 

              precision    recall  f1-score   support

           0       0.62      0.89      0.73       142
           1       0.89      0.63      0.74       208

    accuracy                           0.74       350
   macro avg       0.76      0.76      0.74       350
weighted avg       0.78      0.74      0.74       350

[[126  16]
 [ 76 132]]


In [105]:
#class_report , conf_matrix = Perceptron_classifier(train_vecs, y_train, test_vecs, y_test)
#print('Results of Perceptron Classifier on BOW\n')
#print(class_report)
#print(conf_matrix)

In [106]:
class_report , conf_matrix = KNN_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of KNN_classifier on BOW\n')
print(class_report)
print(conf_matrix)

Results of KNN_classifier on BOW

              precision    recall  f1-score   support

           0       0.96      0.76      0.85       253
           1       0.59      0.91      0.72        97

    accuracy                           0.80       350
   macro avg       0.78      0.84      0.78       350
weighted avg       0.86      0.80      0.81       350

[[193  60]
 [  9  88]]


In [107]:
class_report , conf_matrix = DT_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Decision tree classifier on BOW\n')
print(class_report)
print(conf_matrix)

Results of Decision tree classifier on BOW

              precision    recall  f1-score   support

           0       0.87      0.81      0.84       217
           1       0.72      0.80      0.75       133

    accuracy                           0.80       350
   macro avg       0.79      0.80      0.79       350
weighted avg       0.81      0.80      0.80       350

[[175  42]
 [ 27 106]]


In [108]:
class_report , conf_matrix = XGB_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of XGB classifier  on BOW\n')
print(class_report)
print(conf_matrix)

Results of XGB classifier  on BOW

              precision    recall  f1-score   support

           0       0.92      0.87      0.89       213
           1       0.81      0.88      0.84       137

    accuracy                           0.87       350
   macro avg       0.86      0.87      0.87       350
weighted avg       0.87      0.87      0.87       350

[[185  28]
 [ 17 120]]


# Binary Classification using Bigram/Trigram

In [109]:
vectorizer = CountVectorizer(binary=False,ngram_range=(2,2))
train_vecs = vectorizer.fit_transform(X_train)
test_vecs = vectorizer.transform(X_test)
print (train_vecs.shape, test_vecs.shape)

(710, 250576) (350, 250576)


In [110]:
class_report , conf_matrix = SVM_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of SVM CLASSIFIER on Bi-gram\n')
print(class_report)
print(conf_matrix)

Results of SVM CLASSIFIER on Bi-gram

              precision    recall  f1-score   support

           0       0.97      0.84      0.90       231
           1       0.76      0.94      0.84       119

    accuracy                           0.88       350
   macro avg       0.86      0.89      0.87       350
weighted avg       0.89      0.88      0.88       350

[[195  36]
 [  7 112]]


In [111]:
class_report , conf_matrix = LR_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Logistic Regression Classifier on Bi-gram\n')
print(class_report)
print(conf_matrix)

Results of Logistic Regression Classifier on Bi-gram

              precision    recall  f1-score   support

           0       0.97      0.83      0.90       235
           1       0.74      0.95      0.83       115

    accuracy                           0.87       350
   macro avg       0.85      0.89      0.86       350
weighted avg       0.89      0.87      0.87       350

[[196  39]
 [  6 109]]


In [112]:
class_report , conf_matrix = RF_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of  Random Forest on Bi-gram\n')
print(class_report)
print(conf_matrix)

Results of  Random Forest on Bi-gram

              precision    recall  f1-score   support

           0       0.98      0.79      0.87       249
           1       0.65      0.95      0.77       101

    accuracy                           0.84       350
   macro avg       0.81      0.87      0.82       350
weighted avg       0.88      0.84      0.84       350

[[197  52]
 [  5  96]]


In [113]:
class_report , conf_matrix = NB_classifier(train_vecs.todense(), y_train, test_vecs.todense(), y_test)
print('Results of Naive Bayes Classifier on Bi-gram \n')
print(class_report)
print(conf_matrix)

Results of Naive Bayes Classifier on Bi-gram 

              precision    recall  f1-score   support

           0       0.62      0.95      0.75       132
           1       0.96      0.65      0.78       218

    accuracy                           0.77       350
   macro avg       0.79      0.80      0.77       350
weighted avg       0.83      0.77      0.77       350

[[126   6]
 [ 76 142]]


In [114]:
#class_report , conf_matrix = Perceptron_classifier(train_vecs, y_train, test_vecs, y_test)
#print('Results of Perceptron Classifier on Bi-gram \n')
#print(class_report)
#print(conf_matrix)

In [115]:
class_report , conf_matrix = KNN_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of KNN_classifier on Bi-gram \n')
print(class_report)
print(conf_matrix)

Results of KNN_classifier on Bi-gram 

              precision    recall  f1-score   support

           0       1.00      0.58      0.73       349
           1       0.01      1.00      0.01         1

    accuracy                           0.58       350
   macro avg       0.50      0.79      0.37       350
weighted avg       1.00      0.58      0.73       350

[[202 147]
 [  0   1]]


In [116]:
class_report , conf_matrix = DT_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Decision tree classifier on Bii-gram \n')
print(class_report)
print(conf_matrix)

Results of Decision tree classifier on Bii-gram 

              precision    recall  f1-score   support

           0       0.83      0.78      0.80       216
           1       0.68      0.75      0.71       134

    accuracy                           0.77       350
   macro avg       0.75      0.76      0.76       350
weighted avg       0.77      0.77      0.77       350

[[168  48]
 [ 34 100]]


In [117]:
class_report , conf_matrix = XGB_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of XGB classifier  on Bi-gram \n')
print(class_report)
print(conf_matrix)

Results of XGB classifier  on Bi-gram 

              precision    recall  f1-score   support

           0       0.92      0.85      0.89       218
           1       0.78      0.88      0.83       132

    accuracy                           0.86       350
   macro avg       0.85      0.87      0.86       350
weighted avg       0.87      0.86      0.86       350

[[186  32]
 [ 16 116]]


In [118]:
vectorizer = CountVectorizer(binary=False,ngram_range=(3,3))
train_vecs = vectorizer.fit_transform(X_train)
test_vecs = vectorizer.transform(X_test)
print (train_vecs.shape, test_vecs.shape)

(710, 434230) (350, 434230)


In [119]:
class_report , conf_matrix = SVM_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of SVM CLASSIFIER on Tri-gram\n')
print(class_report)
print(conf_matrix)

Results of SVM CLASSIFIER on Tri-gram

              precision    recall  f1-score   support

           0       1.00      0.70      0.82       286
           1       0.43      0.98      0.59        64

    accuracy                           0.75       350
   macro avg       0.71      0.84      0.71       350
weighted avg       0.89      0.75      0.78       350

[[201  85]
 [  1  63]]


In [120]:
class_report , conf_matrix = LR_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Logistic Regression Classifier on Tri-gram\n')
print(class_report)
print(conf_matrix)

Results of Logistic Regression Classifier on Tri-gram

              precision    recall  f1-score   support

           0       1.00      0.67      0.80       302
           1       0.32      1.00      0.49        48

    accuracy                           0.71       350
   macro avg       0.66      0.83      0.65       350
weighted avg       0.91      0.71      0.76       350

[[202 100]
 [  0  48]]


In [121]:
class_report , conf_matrix = RF_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of  Random Forest on Tri-gram\n')
print(class_report)
print(conf_matrix)

Results of  Random Forest on Tri-gram

              precision    recall  f1-score   support

           0       1.00      0.64      0.78       316
           1       0.23      1.00      0.37        34

    accuracy                           0.67       350
   macro avg       0.61      0.82      0.58       350
weighted avg       0.93      0.67      0.74       350

[[202 114]
 [  0  34]]


In [122]:
class_report , conf_matrix = NB_classifier(train_vecs.todense(), y_train, test_vecs.todense(), y_test)
print('Results of Naive Bayes Classifier on Tri-gram \n')
print(class_report)
print(conf_matrix)

Results of Naive Bayes Classifier on Tri-gram 

              precision    recall  f1-score   support

           0       0.60      0.95      0.74       127
           1       0.96      0.64      0.77       223

    accuracy                           0.75       350
   macro avg       0.78      0.79      0.75       350
weighted avg       0.83      0.75      0.75       350

[[121   6]
 [ 81 142]]


In [123]:
#class_report , conf_matrix = Perceptron_classifier(train_vecs, y_train, test_vecs, y_test)
#print('Results of Perceptron Classifier on Tri-gram \n')
#print(class_report)
#print(conf_matrix)

In [124]:
class_report , conf_matrix = KNN_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of KNN_classifier on Tri-gram \n')
print(class_report)
print(conf_matrix)

Results of KNN_classifier on Tri-gram 

              precision    recall  f1-score   support

           0       1.00      0.58      0.73       350
           1       0.00      0.00      0.00         0

    accuracy                           0.58       350
   macro avg       0.50      0.29      0.37       350
weighted avg       1.00      0.58      0.73       350

[[202 148]
 [  0   0]]


In [125]:
class_report , conf_matrix = DT_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of Decision tree classifier on Tri-gram \n')
print(class_report)
print(conf_matrix)

Results of Decision tree classifier on Tri-gram 

              precision    recall  f1-score   support

           0       0.86      0.73      0.79       237
           1       0.57      0.74      0.64       113

    accuracy                           0.73       350
   macro avg       0.71      0.74      0.72       350
weighted avg       0.76      0.73      0.74       350

[[173  64]
 [ 29  84]]


In [126]:
class_report , conf_matrix = XGB_classifier(train_vecs, y_train, test_vecs, y_test)
print('Results of XGB classifier  on Tri-gram \n')
print(class_report)
print(conf_matrix)

Results of XGB classifier  on Tri-gram 

              precision    recall  f1-score   support

           0       0.90      0.81      0.85       224
           1       0.71      0.83      0.77       126

    accuracy                           0.82       350
   macro avg       0.80      0.82      0.81       350
weighted avg       0.83      0.82      0.82       350

[[181  43]
 [ 21 105]]
